In [1]:
#EDIT ME to change target subreddit...
target_subreddit = "comicbooks" # scraped data includes: ['Naruto', 'BokuNoHeroAcademia', 'anime', 'manga', 'comicbooks',
                                                    # 'StarWars', 'startrek', 'LonghornNation', 'cfb', 'Genshin_Impact']
scrape_type = "submission"  # set to "submission" or "comment"

# Import

In [2]:
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
# from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
# Read in data
all_data = pd.read_csv(f'./data/{target_subreddit}_{scrape_type}s.csv')

In [4]:
all_data.head()

,Unnamed: 0,subreddit,selftext,author_fullname,gilded,is_gallery,title,link_flair_richtext,subreddit_name_prefixed,hidden,...,retrieved_utc,updated_utc,utc_datetime_str,post_hint,crosspost_parent_list,preview,link_flair_template_id,crosspost_parent,author_cakeday,edited_on
0,0,comicbooks,NaN,t2_a0obk3gv,0,True,Planet Hulk body used to create The Maxx,[],r/comicbooks,False,...,1677990867,1677990868,2023-03-05 04:34:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,comicbooks,NaN,t2_112uoc,0,NaN,[OC] Frontier Graveyard has a new episode.,"[{'e': 'text', 't': 'News'}]",r/comicbooks,False,...,1677990594,1677990594,2023-03-05 04:29:42,image,"[{'approved_at_utc': None, 'subreddit': 'zombi...",{'images': [{'source': {'url': 'https://previe...,e00f666e-4b9c-11e3-8795-12313b074434,t3_11ilf8q,NaN,NaN
2,2,comicbooks,Starting my second box and would prefer not ge...,t2_68d44nnp,0,NaN,Any recommendations on quality storage boxes?,"[{'e': 'text', 't': 'Question'}]",r/comicbooks,False,...,1677990059,1677990060,2023-03-05 04:20:47,NaN,NaN,NaN,313e6c50-bc89-11e1-ae6a-12313d14a568,NaN,NaN,NaN
3,3,comicbooks,NaN,t2_j0ubh1u,0,True,Favorite comic book cameos from characters out...,"[{'e': 'text', 't': 'Excerpt'}]",r/comicbooks,False,...,1677984267,1677984267,2023-03-05 02:44:13,NaN,NaN,NaN,0583ca00-b4a1-11e1-ae43-12313b0c247a,NaN,NaN,NaN
4,4,comicbooks,NaN,t2_75sge4r3,0,NaN,Got these Two off of Amazon today,"[{'e': 'text', 't': 'Other'}]",r/comicbooks,False,...,1677983783,1677983784,2023-03-05 02:36:11,image,NaN,{'images': [{'source': {'url': 'https://previe...,cbfa3ab2-4bc1-11e3-9fcd-12313b04ceaf,NaN,NaN,NaN


# Choosing a sub-set of columns worth keeping for (possible) analysis

In [5]:
all_data.columns

Index(['Unnamed: 0', 'subreddit', 'selftext', 'author_fullname', 'gilded',
       'is_gallery', 'title', 'link_flair_richtext', 'subreddit_name_prefixed',
       'hidden', 'pwls', 'link_flair_css_class', 'thumbnail_height',
       'top_awarded_type', 'hide_score', 'media_metadata', 'quarantine',
       'link_flair_text_color', 'upvote_ratio',
       'author_flair_background_color', 'domain', 'media_embed',
       'thumbnail_width', 'author_flair_template_id', 'is_original_content',
       'secure_media', 'is_reddit_media_domain', 'is_meta', 'category',
       'secure_media_embed', 'gallery_data', 'link_flair_text', 'score',
       'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited',
       'author_flair_css_class', 'author_flair_richtext', 'gildings',
       'content_categories', 'is_self', 'subreddit_type', 'link_flair_type',
       'wls', 'removed_by_category', 'author_flair_type',
       'total_awards_received', 'allow_live_comments', 'suggested_sort',
       'url_over

In [6]:
# columns investigated one-by-one, systematically... no real valuable data found other than columns used below

# all_data['gildings'].value_counts()
# all_data[all_data['gildings']!="{}"]

In [7]:
df = all_data[['subreddit','selftext', 'author_fullname', 'title', 'upvote_ratio', 
              'is_reddit_media_domain', 'domain', 'over_18', 'spoiler', 'id', 
              'author', 'num_comments', 'created_utc', 'is_video'
             ]].copy()   #used .copy() to reduce number of warnings later

df.head()

,subreddit,selftext,author_fullname,title,upvote_ratio,is_reddit_media_domain,domain,over_18,spoiler,id,author,num_comments,created_utc,is_video
0,comicbooks,NaN,t2_a0obk3gv,Planet Hulk body used to create The Maxx,1.0,False,reddit.com,False,False,11inl15,Hairy-Ad9897,0,1677990850,False
1,comicbooks,NaN,t2_112uoc,[OC] Frontier Graveyard has a new episode.,1.0,True,i.redd.it,False,False,11inhqq,profesone,0,1677990582,False
2,comicbooks,Starting my second box and would prefer not ge...,t2_68d44nnp,Any recommendations on quality storage boxes?,1.0,False,self.comicbooks,False,False,11inb9l,HenjaminBenry,0,1677990047,False
3,comicbooks,NaN,t2_j0ubh1u,Favorite comic book cameos from characters out...,1.0,False,reddit.com,False,False,11ilad6,Christianduty,0,1677984253,False
4,comicbooks,NaN,t2_75sge4r3,Got these Two off of Amazon today,1.0,True,i.redd.it,False,False,11il46k,NinjaZero2099,0,1677983771,False


# Create new columns that might be useful

In [8]:
# Convert UTC times to human-readable

df['created_datetime'] = pd.to_datetime(df['created_utc'], unit='s')

#Reference: https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [9]:
# Make a combined column of seltext + title, for combined processing

cond1 = df['selftext'].notna()
cond2 = df['selftext'] != '[removed]'

df['combined_text'] = np.where((cond1 & cond2),
                                df['title'] + " " + df['selftext'],
                                df['title'])      #if selftext is blank or [removed], just use only the title

In [10]:
df["combined_text_length"] = df['combined_text'].str.len()

In [11]:
df["combined_wordcount"] = df['combined_text'].str.split(" ").str.len()

# Text normalization (lemmatizing, tokenizing, stop words)

In [12]:
# Lemmatizing 
# Reference: Lesson 504

wn = WordNetLemmatizer()

def custom_lemmatize(word, tag):
    mapper = {
        'J': wordnet.ADJ,
        'V': wordnet.VERB,
        'N': wordnet.NOUN,
        'R': wordnet.ADV
    }
    pos = mapper.get(tag[0])
    
    return wn.lemmatize(word, pos) if pos else word

df['lemmatized_text'] = ""

for i in range(df.shape[0]):
    df['lemmatized_text'][i] = \
        " ".join([custom_lemmatize(word, tag) for word, tag in nltk.pos_tag( df['combined_text'][i].split(" "))])
    
    #Runs slow & gives a warning, but works

C:\Users\panda\AppData\Local\Temp\ipykernel_22172\3621847600.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemmatized_text'][i] = \


In [13]:
# Tokenizing & stopword removal AFTER lemmatizing
# Reference: Lesson 504

tokenizer = RegexpTokenizer('\w+')

df['tokenized_text'] = ""

for i in range(df.shape[0]):
    my_text_tokens = tokenizer.tokenize(df['lemmatized_text'][i].lower())
    my_text_tokens_ns = [token for token in my_text_tokens if token not in stopwords.words("english")]
    recombined_text = " ".join(my_text_tokens_ns)
    df['tokenized_text'][i] = recombined_text
    
    #Runs slow & gives a warning, but works

C:\Users\panda\AppData\Local\Temp\ipykernel_22172\3578492394.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokenized_text'][i] = recombined_text


In [14]:
df.head()

,subreddit,selftext,author_fullname,title,upvote_ratio,is_reddit_media_domain,domain,over_18,spoiler,id,author,num_comments,created_utc,is_video,created_datetime,combined_text,combined_text_length,combined_wordcount,lemmatized_text,tokenized_text
0,comicbooks,NaN,t2_a0obk3gv,Planet Hulk body used to create The Maxx,1.0,False,reddit.com,False,False,11inl15,Hairy-Ad9897,0,1677990850,False,2023-03-05 04:34:10,Planet Hulk body used to create The Maxx,40,8,Planet Hulk body use to create The Maxx,planet hulk body use create maxx
1,comicbooks,NaN,t2_112uoc,[OC] Frontier Graveyard has a new episode.,1.0,True,i.redd.it,False,False,11inhqq,profesone,0,1677990582,False,2023-03-05 04:29:42,[OC] Frontier Graveyard has a new episode.,42,7,[OC] Frontier Graveyard have a new episode.,oc frontier graveyard new episode
2,comicbooks,Starting my second box and would prefer not ge...,t2_68d44nnp,Any recommendations on quality storage boxes?,1.0,False,self.comicbooks,False,False,11inb9l,HenjaminBenry,0,1677990047,False,2023-03-05 04:20:47,Any recommendations on quality storage boxes? ...,200,33,Any recommendation on quality storage boxes? S...,recommendation quality storage boxes starting ...
3,comicbooks,NaN,t2_j0ubh1u,Favorite comic book cameos from characters out...,1.0,False,reddit.com,False,False,11ilad6,Christianduty,0,1677984253,False,2023-03-05 02:44:13,Favorite comic book cameos from characters out...,61,9,Favorite comic book cameo from character outsi...,favorite comic book cameo character outside co...
4,comicbooks,NaN,t2_75sge4r3,Got these Two off of Amazon today,1.0,True,i.redd.it,False,False,11il46k,NinjaZero2099,0,1677983771,False,2023-03-05 02:36:11,Got these Two off of Amazon today,33,7,Got these Two off of Amazon today,got two amazon today


In [15]:
df.shape

(4845, 20)

# Write output to csv

In [16]:
df.to_csv(f'./data/{target_subreddit}_{scrape_type}s_preprocessed.csv', index=False)